In [11]:
import requests
import ipywidgets as widgets
import json
import time
from IPython.display import display, HTML
from traitlets import Int, Unicode, observe

In [12]:
display(HTML('<link rel="stylesheet" type="text/css" href="//igv.org/web/release/1.0.6/igv-1.0.6.css">'))

In [13]:
class TabsWidget(widgets.DOMWidget):
    
    _view_name = Unicode('TabsView').tag(sync=True)
    _view_module = Unicode('tabsDemo').tag(sync=True)
    frameHeight = Int(300).tag(sync=True)
    _chromLocString = "chr1:0-0";
    _rawMessage = "";
    _incomingMessage = {};
    msgFromKernel = Unicode("{}").tag(sync=True)
    msgFromBrowser = Unicode("{}").tag(sync=True)
    status = "initial status message\n"

       # ensure that any ensuing message is seen as novel in the browser
       # the message (a json-ified string) is a mutable traitlet.  the browser only sees 
       # the message if it changes
    def _sendResetMessageToBrowser(self):  
       self.msgFromKernel = json.dumps({"cmd": "cleanSlate", "status": "nop", "callback": "", "payload": ""});

    def setHeight(self, height):
       print("setHeight(%d) "% height)
       self.frameHeight = height
        
    def getChromLocString(self):
        return self._chromLocString
    
    def ping(self):
        self._sendResetMessageToBrowser()
        self.msgFromKernel = json.dumps({"cmd": "ping", "status": "request", "callback": "", "payload": ""});

    def deleteGraph(self):
      self._sendResetMessageToBrowser()
      self.msgFromKernel = json.dumps({"cmd": "deleteGraph", "status": "request", "callback": "", "payload": ""});

    def addGraph(self, gjson):
      self._sendResetMessageToBrowser();
      self.msgFromKernel = json.dumps({"cmd": "addGraph", "status": "request",
                                       "callback": "",    "payload": gjson});


    @observe('msgFromBrowser')
    def msg_arrived(self, change):
       #self.status += "msgFromBrowser has arrived: %f\n" % time.time()
       self._rawMessage = change['new']
       #self.status += "rawMessage: %s\n" % self._rawMessage
       self._incomingMessage = json.loads(self._rawMessage)
       cmd = self._incomingMessage["cmd"]
       #self.status += "cmd: %s\n"  % cmd
       if(cmd == "updateChromLocString"):
          self._chromLocString = self._incomingMessage["payload"]
       else:
         self.status += "unrecognized cmd: '%s'" % cmd


display(HTML(data="""
<style>
    div#notebook-container    { width: 97%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [14]:
%%javascript
"use strict"
require.config({
    shim: {'bootstrap': {'deps' :['jquery']},
           'igv':       {'deps' :['jquery', 'jquery-ui', 'bootstrap']}
           },

    paths: {'jquery'    :   'http://code.jquery.com/jquery-1.12.4.min',
            'jquery-ui' :   'http://code.jquery.com/ui/1.12.1/jquery-ui.min',
            'cytoscape' :   'http://cytoscape.github.io/cytoscape.js/api/cytoscape.js-latest/cytoscape',
            //'cytoscape' :   'http://localhost:8099/js/cytoscape-2.7.10',
            'bootstrap' :   'http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min',
            'igv'       :   'http://igv.org/web/release/1.0.6/igv-1.0.6'
           }
      });

require.undef('tabsDemo')

define('tabsDemo', ["jupyter-js-widgets", "jquery", "jquery-ui", "cytoscape", "igv"], 
       function(widgets, $, ui, cytoscape, igv) {
    
    var TabsView = widgets.DOMWidgetView.extend({

        initialize: function() {
           this.options = {}
           this.chromLocString = "";
           console.log("constructing TabsView");
           this.frameHeight = "800px";
           },

        resizeHandler: function(){
           console.log("TabsView resizeHandler")            
           },
        
        createMasterTabsDiv: function(){
           var masterTabsDiv = $("<div id='masterTabsDiv' style='border:1px solid gray; height: 1200px; width: 97%'></div>");
           var list = $("<ul/>");
           list.append("<li><a href='#tab-2'>igv</a></li>");
           list.append("<li><a href='#tab-1'>cytoscape</a></li>");
           list.append("<li><a href='#tab-3'>three</a></li>");
           masterTabsDiv.append(list);

           var tab2 = $("<div id='tab-1'></div>");
           tab2.append("<div id='cyDiv' style='border:1px solid blue; height:1000px;'></div>");

           var tab1 = $("<div id='tab-2'></div>");
           tab1.append("<div id='igvDiv' style='border:1px solid green; height:1000px;'></div>");
            
            
           var tab3 = $("<div id='tab-3'>contents 3</div>");
            
           masterTabsDiv.append(tab1);
           masterTabsDiv.append(tab2);
           masterTabsDiv.append(tab3);
            
           return(masterTabsDiv);
           },
         
        getCyOptions: function(){
           var value = {container: $("#cyDiv"), 
                                      elements: {nodes: [{data: {id:'a'}}],
                                                 edges: [{data:{source:'a', target:'a'}}]},
                                      style: cytoscape.stylesheet()
                                        .selector('node').style({'background-color': '#d22',
                                                                 'label': 'data(id)',
                                                                 'text-valign': 'center',
                                                                 'text-halign': 'center',
                                                                 'border-width': 1})
                                         .selector('edge').style({'line-color': 'black',
                                                                  'target-arrow-shape': 'triangle',
                                                                  'target-arrow-color': 'black',
                                                                  'curve-style': 'bezier'})
                       };
            var emptyGraphValue = {container:$("#cyDiv")};
            //return(emptyGraphValue);
            return(value);
            }, // getCyOptions
        
        
        getIgvOptions: function(){
           var igvOptionsLocal = {locus: "5:88,621,548-88,999,827",
             reference: {id: "geneSymbols_hg38",
                fastaURL: "http://localhost/data/genomes/human_g1k_v37_decoy.fasta",
                 cytobandURL: "http://localhost/data/annotations/b37_cytoband.txt"
                 },
                tracks: [
                  {name: 'Gencode v24',
                   url: "http://localhost://data/hg38/gencode.v24.annotation.sorted.gtf.gz",
                  indexURL: "http://localhost://data/hg38/gencode.v24.annotation.sorted.gtf.gz.tbi",
                  format: 'gtf',
                  visibilityWindow: 2000000,
                  displayMode: 'EXPANDED'
                  },
                 {name: 'geneSymbols_hg38',
                  url: 'http://localhost/data/hg38/geneSymbolSearch.bed',
                  indexed: false,
                  searchable: true,
                  //visibilityWindow: 5000000,
                  displayMode: 'COLLAPSED',
                  color: "#448844"
                  },
                 {name: "igap gwas",
                  url: 'http://localhost/data/hg38/variants/igap.bed',
                  indexed: false,
                  searchable: true,
                  //visibilityWindow: 5000000,
                  displayMode: 'EXPANDED',
                  color: "#884444"
                  }
                  ]
              }; // igvOptionsLocal
             return(igvOptionsLocal);
           }, // getIgvOptions
 
        render: function() {
            var multiWidget = this;
            console.log("entering render");
            this.listenTo(this.model, 'change:msgFromKernel', this.dispatchRequest, this);
            this.masterTabsDiv = this.createMasterTabsDiv();
            this.$el.append(this.masterTabsDiv);
            this.listenTo(this.model, 'change:frameHeight', this.frameDimensionsChanged, this);
            setTimeout(function(){
               window.browser = igv.createBrowser($("#igvDiv"), multiWidget.getIgvOptions());
               //window.browser.on('locuschange', function(referenceFrame, chromLocString){multiWidget.chromLocString=chromLocString});
               window.browser.on('locuschange', function(referenceFrame, chromLocString){
                   multiWidget.updateChromLocStringToKernel(multiWidget, chromLocString)});
               setTimeout(function() {
                   console.log("about to call tabs()");
                    $("#masterTabsDiv").tabs({
                       activate: function(){
                         console.log("tabs activate")
                         cy.resize();
                       }});
                   $("[href='#tab-2']").trigger("click");
                   }, 8000);
               }, 3000);
            setTimeout(function(){
               var options = multiWidget.getCyOptions();
               console.log(JSON.stringify(options));
               window.cy = cytoscape(multiWidget.getCyOptions());
               console.log("window.cy created")
               }, 1000);
             }, // render

        dispatchRequest: function(){
           console.log("dispatchRequest");
           var msgRaw = this.model.get("msgFromKernel");
           var msg = JSON.parse(msgRaw);
           console.log(msg);
           console.log("========================");
           console.log(this);
           switch(msg.cmd) {
              case 'ping':
                console.log("ping! from the kernel");
                break;
              case 'deleteGraph':
                cy.edges().remove();
                cy.nodes().remove();
                break;
              case 'addGraph':
                var jsonGraph = msg.payload;
                cy.json(jsonGraph);
                break;
            default:
               console.log("unrecognized msg.cmd: " + msg.cmd);
             } // switch
           }, 




       updateChromLocStringToKernel: function(multiWidget, newString){
           console.log("xxx " + newString);
           var jsonString = JSON.stringify({cmd: "updateChromLocString",
                                            status: "request",
                                            callback: "",
                                            payload: newString});
            multiWidget.model.set("msgFromBrowser", jsonString);
            multiWidget.touch()
            },
                                                


        frameDimensionsChanged: function(){
           console.log("frameDimensionsChanged");
           var oldHeight = $("#mainDiv").height()
           var oldWidth = $("#mainDiv").width()
           var newHeight = this.model.get("frameHeight");
           var msg = "<center>tabs demo, height: " + oldHeight + " -> " + newHeight + "</center>";
           $("#mainDiv").html(msg);
           $("#masterTabsDiv").height(newHeight);
           }, 
    });
    return {
        TabsView: TabsView
    };
});

<IPython.core.display.Javascript object>

In [15]:
app = TabsWidget()
display(app)

In [16]:
app.deleteGraph()

In [17]:
gjsonText = "{\"data\":{},\"elements\":{\"nodes\":[{\"data\":{\"name\":\"VGF\",\"type\":\"targetGene\",\"label\":\"VGF\",\"distance\":0,\"gene.cor\":0,\"beta\":0,\"purity\":0,\"id\":\"VGF\"}},{\"data\":{\"name\":\"MAZ\",\"type\":\"TF\",\"label\":\"MAZ\",\"distance\":0,\"gene.cor\":0.3677,\"beta\":0.3677,\"purity\":369.3957,\"id\":\"MAZ\"}},{\"data\":{\"name\":\"NRF1\",\"type\":\"TF\",\"label\":\"NRF1\",\"distance\":0,\"gene.cor\":-0.2952,\"beta\":-0.2952,\"purity\":400.4611,\"id\":\"NRF1\"}},{\"data\":{\"name\":\"VGF.fp.upstream.00009\",\"type\":\"footprint\",\"label\":\"9\",\"distance\":9,\"gene.cor\":0,\"beta\":0,\"purity\":0,\"id\":\"VGF.fp.upstream.00009\"}},{\"data\":{\"name\":\"VGF.fp.downstream.00018\",\"type\":\"footprint\",\"label\":\"-18\",\"distance\":-18,\"gene.cor\":0,\"beta\":0,\"purity\":0,\"id\":\"VGF.fp.downstream.00018\"}}],\"edges\":[{\"data\":{\"source\":\"MAZ\",\"target\":\"VGF.fp.upstream.00009\",\"edgeType\":\"bindsTo\",\"beta\":0,\"purity\":0,\"id\":\"6789bfce-ca11-4555-9f69-f02fba0cd911\"}},{\"data\":{\"source\":\"NRF1\",\"target\":\"VGF.fp.downstream.00018\",\"edgeType\":\"bindsTo\",\"beta\":0,\"purity\":0,\"id\":\"877069f1-76ab-4d57-8528-d00816159066\"}},{\"data\":{\"source\":\"VGF.fp.upstream.00009\",\"target\":\"VGF\",\"edgeType\":\"regulatorySiteFor\",\"beta\":0,\"purity\":0,\"id\":\"31897eac-da0c-4cb4-bd78-62f16bcdd53f\"}},{\"data\":{\"source\":\"VGF.fp.downstream.00018\",\"target\":\"VGF\",\"edgeType\":\"regulatorySiteFor\",\"beta\":0,\"purity\":0,\"id\":\"3a08387b-23d6-4ebe-8ce5-3b781bd67daa\"}}]}}"
gjson = json.loads(gjsonText)

In [18]:
app.addGraph(gjson)

In [9]:
x2= app.getChromLocString(); print(x2)

chr1:0-0


In [10]:
for i in range(10):
     app.ping()

In [11]:
app.ping()

In [12]:
import zmq

In [ ]:
socketContext = zmq.Context()
socket = socketContext.socket(zmq.REQ)
socket.connect("tcp://whovian:%s" % '5557')

In [ ]:
for i in range(5):
   socket.send_string("hello from python %d" % i)
   print(socket.recv_string())


In [ ]:
msg =